# AutoML - Classification

PyCaret
* Main Site - https://pycaret.org/
* Docs - https://pycaret.readthedocs.io/en/latest/

## Table of Contents

* [Setup and Preprocessing](#setup_cls)  
* [Compare Models](#compare_cls)  
* [Create Model](#create_cls)  
* [Tune Model](#tune_cls)  
* [Evaluate Model](#evaluate_cls)  
* [Finalize and Store Model](#finalize_and_store_cls)

## Imports and Global Settings

In [10]:
import os
import sys
import datetime
import json
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sklearn.metrics import accuracy_score, precision_score
from pycaret.classification import ClassificationExperiment

here = os.getcwd()
sys.path.append(os.path.join(here, ".."))

from src.utils.modeling_utils import ModelSetup, calculate_roi, save_model_report

load_dotenv()
RDS_ENDPOINT = os.getenv("RDS_ENDPOINT")
RDS_PASSWORD = os.getenv("RDS_PASSWORD")

# Pandas Settings
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 1000)
pd.options.display.max_info_columns = 200
pd.options.display.precision = 5

## Load Data

In [11]:
username = "postgres"
password = RDS_PASSWORD
endpoint = RDS_ENDPOINT
database = "nba_betting"
port = "5432"

# Create the connection string
connection_string = (
    f"postgresql+psycopg2://{username}:{password}@{endpoint}:{port}/{database}"
)

### Games

In [12]:
start_date = "2020-09-01"
start_date_int = int(start_date.replace("-", ""))  # Convert date to YYYYMMDD format

features = [
    "game_id",
    "game_datetime",
    "home_team",
    "away_team",
    "open_line",
    "home_score",
    "away_score",
    "game_completed",
    "odds_last_update",
    "scores_last_update",
]

# Extracting the YYYYMMDD portion of the game_id and comparing it with start_date_int
games_query = f"SELECT {', '.join(features)} FROM games WHERE CAST(LEFT(game_id, 8) AS INTEGER) >= {start_date_int};"

with create_engine(connection_string).connect() as connection:
    games = pd.read_sql_query(games_query, connection)

### Features

In [13]:
start_date = "2020-09-01"
start_date_int = int(start_date.replace("-", ""))  # Convert date to YYYYMMDD format

features = ["game_id", "data"]

# Extracting the YYYYMMDD portion of the game_id and comparing it with start_date_int
features_query = f"SELECT {', '.join(features)} FROM all_features_json WHERE CAST(LEFT(game_id, 8) AS INTEGER) >= {start_date_int};"

with create_engine(connection_string).connect() as connection:
    all_features = pd.read_sql_query(features_query, connection)

# Normalize the JSON strings in the 'data' column
expanded_data = pd.json_normalize(all_features["data"])

# Drop the original 'data' column and concatenate the expanded data
all_features = pd.concat([all_features.drop(columns=["data"]), expanded_data], axis=1)

In [14]:
games_features = pd.merge(
    games,
    all_features,
    on="game_id",
    how="left",
    validate="one_to_one",
    suffixes=("", "_drop"),
)
# Drop the columns from df2 (with suffix '_drop')
games_features = games_features[
    games_features.columns.drop(list(games_features.filter(regex="_drop")))
]

<a id='basic_data_overview'></a>

## Basic Data Overview

In [15]:
df = games_features.copy()

In [16]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3956 entries, 0 to 3955
Data columns (total 715 columns):
 #    Column                                        Non-Null Count  Dtype         
---   ------                                        --------------  -----         
 0    game_id                                       3956 non-null   object        
 1    game_datetime                                 3956 non-null   datetime64[ns]
 2    home_team                                     3956 non-null   object        
 3    away_team                                     3956 non-null   object        
 4    open_line                                     3853 non-null   float64       
 5    home_score                                    3855 non-null   float64       
 6    away_score                                    3855 non-null   float64       
 7    game_completed                                3955 non-null   object        
 8    odds_last_update                              101 non-nu

In [17]:
df.head(10)

,game_id,game_datetime,home_team,away_team,open_line,home_score,away_score,game_completed,odds_last_update,scores_last_update,season,elo1_pre,elo2_pre,elo_prob1,elo_prob2,last_5_hv,streak_hv,win_pct_hv,raptor1_pre,raptor2_pre,season_type,raptor_prob1,raptor_prob2,rest_diff_hv,carm_elo1_pre,carm_elo2_pre,day_of_season,point_diff_hv,carm_elo_prob1,carm_elo_prob2,away_team_last_5,away_team_streak,home_team_last_5,home_team_streak,away_team_win_pct,home_team_win_pct,point_diff_last_5_hv,pie_away_all_advanced,pie_away_l2w_advanced,pie_home_all_advanced,pie_home_l2w_advanced,l_away_all_traditional,l_away_l2w_traditional,l_home_all_traditional,l_home_l2w_traditional,pace_away_all_advanced,pace_away_l2w_advanced,pace_home_all_advanced,pace_home_l2w_advanced,poss_away_all_advanced,poss_away_l2w_advanced,poss_home_all_advanced,poss_home_l2w_advanced,w_away_all_traditional,w_away_l2w_traditional,w_home_all_traditional,w_home_l2w_traditional,gp_away_all_traditional,gp_away_l2w_traditional,gp_home_all_traditional,gp_home_l2w_traditional,ast_to_away_all_advanced,ast_to_away_l2w_advanced,ast_to_home_all_advanced,ast_to_home_l2w_advanced,away_team_avg_point_diff,e_pace_away_all_advanced,e_pace_away_l2w_advanced,e_pace_home_all_advanced,e_pace_home_l2w_advanced,home_team_avg_point_diff,min_away_all_traditional,min_away_l2w_traditional,min_home_all_traditional,min_home_l2w_traditional,opp_pf_away_all_opponent,opp_pf_away_l2w_opponent,opp_pf_home_all_opponent,opp_pf_home_l2w_opponent,pts_away_all_traditional,pts_away_l2w_traditional,pts_home_all_traditional,pts_home_l2w_traditional,ts_pct_away_all_advanced,ts_pct_away_l2w_advanced,ts_pct_home_all_advanced,ts_pct_home_l2w_advanced,ast_pct_away_all_advanced,ast_pct_away_l2w_advanced,ast_pct_home_all_advanced,ast_pct_home_l2w_advanced,days_since_last_game_away,days_since_last_game_home,efg_pct_away_all_advanced,efg_pct_away_l2w_advanced,efg_pct_home_all_advanced,efg_pct_home_l2w_advanced,opp_ast_away_all_opponent,opp_ast_away_l2w_opponent,opp_ast_home_all_opponent,opp_ast_home_l2w_opponent,opp_blk_away_all_opponent,opp_blk_away_l2w_opponent,opp_blk_home_all_opponent,opp_blk_home_l2w_opponent,opp_fga_away_all_opponent,opp_fga_away_l2w_opponent,opp_fga_home_all_opponent,opp_fga_home_l2w_opponent,opp_fgm_away_all_opponent,opp_fgm_away_l2w_opponent,opp_fgm_home_all_opponent,opp_fgm_home_l2w_opponent,opp_fta_away_all_opponent,opp_fta_away_l2w_opponent,opp_fta_home_all_opponent,opp_fta_home_l2w_opponent,opp_ftm_away_all_opponent,opp_ftm_away_l2w_opponent,opp_ftm_home_all_opponent,opp_ftm_home_l2w_opponent,opp_pfd_away_all_opponent,opp_pfd_away_l2w_opponent,opp_pfd_home_all_opponent,opp_pfd_home_l2w_opponent,opp_pts_away_all_opponent,opp_pts_away_l2w_opponent,opp_pts_home_all_opponent,opp_pts_home_l2w_opponent,opp_reb_away_all_opponent,opp_reb_away_l2w_opponent,opp_reb_home_all_opponent,opp_reb_home_l2w_opponent,opp_stl_away_all_opponent,opp_stl_away_l2w_opponent,opp_stl_home_all_opponent,opp_stl_home_l2w_opponent,opp_tov_away_all_opponent,opp_tov_away_l2w_opponent,opp_tov_home_all_opponent,opp_tov_home_l2w_opponent,reb_pct_away_all_advanced,reb_pct_away_l2w_advanced,reb_pct_home_all_advanced,reb_pct_home_l2w_advanced,dreb_pct_away_all_advanced,dreb_pct_away_l2w_advanced,dreb_pct_home_all_advanced,dreb_pct_home_l2w_advanced,opp_blka_away_all_opponent,opp_blka_away_l2w_opponent,opp_blka_home_all_opponent,opp_blka_home_l2w_opponent,opp_dreb_away_all_opponent,opp_dreb_away_l2w_opponent,opp_dreb_home_all_opponent,opp_dreb_home_l2w_opponent,opp_fg3a_away_all_opponent,opp_fg3a_away_l2w_opponent,opp_fg3a_home_all_opponent,opp_fg3a_home_l2w_opponent,opp_fg3m_away_all_opponent,opp_fg3m_away_l2w_opponent,opp_fg3m_home_all_opponent,opp_fg3m_home_l2w_opponent,opp_oreb_away_all_opponent,opp_oreb_away_l2w_opponent,opp_oreb_home_all_opponent,opp_oreb_home_l2w_opponent,oreb_pct_away_all_advanced,oreb_pct_away_l2w_advanced,oreb_pct_home_all_advanced,oreb_pct_home_l2w_advanced,w_pct_away_all_traditional,w_pct_away_l2w_trad

## Data Preparation

#### Drop Non-Completed Games and Games with No Line

In [18]:
df = df[df["game_completed"] == True]
df = df.dropna(subset=["open_line"])

In [19]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3852 entries, 67 to 3955
Data columns (total 715 columns):
 #    Column                                        Non-Null Count  Dtype         
---   ------                                        --------------  -----         
 0    game_id                                       3852 non-null   object        
 1    game_datetime                                 3852 non-null   datetime64[ns]
 2    home_team                                     3852 non-null   object        
 3    away_team                                     3852 non-null   object        
 4    open_line                                     3852 non-null   float64       
 5    home_score                                    3852 non-null   float64       
 6    away_score                                    3852 non-null   float64       
 7    game_completed                                3852 non-null   object        
 8    odds_last_update                              0 non-nul

### Create Targets

In [20]:
df = ModelSetup.add_targets(df)

### Select Features

In [21]:
training_dates, testing_dates = ModelSetup.choose_dates([2020, 2021], [2022], "Reg")
print("Training Dates:")
print(training_dates)
print("Testing Dates:")
print(testing_dates)

Training Dates:
('2020-12-22', '2022-04-10')
Testing Dates:
('2022-10-18', '2023-04-09')


In [22]:
for col in df.columns:
    print(col)

game_id
game_datetime
home_team
away_team
open_line
home_score
away_score
game_completed
odds_last_update
scores_last_update
season
elo1_pre
elo2_pre
elo_prob1
elo_prob2
last_5_hv
streak_hv
win_pct_hv
raptor1_pre
raptor2_pre
season_type
raptor_prob1
raptor_prob2
rest_diff_hv
carm_elo1_pre
carm_elo2_pre
day_of_season
point_diff_hv
carm_elo_prob1
carm_elo_prob2
away_team_last_5
away_team_streak
home_team_last_5
home_team_streak
away_team_win_pct
home_team_win_pct
point_diff_last_5_hv
pie_away_all_advanced
pie_away_l2w_advanced
pie_home_all_advanced
pie_home_l2w_advanced
l_away_all_traditional
l_away_l2w_traditional
l_home_all_traditional
l_home_l2w_traditional
pace_away_all_advanced
pace_away_l2w_advanced
pace_home_all_advanced
pace_home_l2w_advanced
poss_away_all_advanced
poss_away_l2w_advanced
poss_home_all_advanced
poss_home_l2w_advanced
w_away_all_traditional
w_away_l2w_traditional
w_home_all_traditional
w_home_l2w_traditional
gp_away_all_traditional
gp_away_l2w_traditional
gp_home_a

In [23]:
features_to_use = ["open_line"]

In [24]:
training_df, testing_df, model_report = ModelSetup.create_datasets(
    df, "cls", features_to_use, training_dates, testing_dates, create_report=True
)

In [25]:
print("Training Shape: ", training_df.shape)
print("Testing Shape: ", testing_df.shape)

Training Shape:  (2399, 3)
Testing Shape:  (1230, 3)


### Baselines

In [26]:
training_baseline_home_team = model_report["ind_baseline_train"]
testing_baseline_home_team = model_report["ind_baseline_test"]

training_baseline_majority_class = model_report["dep_baseline_train"]
testing_baseline_majority_class = model_report["dep_baseline_test"]

print(f"Training Baseline - Home Team: {training_baseline_home_team:.2f}")
print(f"Testing Baseline - Home Team: {testing_baseline_home_team:.2f}")
print(f"Training Baseline - Majority Class: {training_baseline_majority_class:.2f}")
print(f"Testing Baseline - Majority Class: {testing_baseline_majority_class:.2f}")

Training Baseline - Home Team: 0.48
Testing Baseline - Home Team: 0.50
Training Baseline - Majority Class: 0.52
Testing Baseline - Majority Class: 0.50


<a id=Classification></a>

## Classification

<a id=setup_cls></a>

### Setup and Preprocessing

In [27]:
py_cls = ClassificationExperiment()

The setup process involves a lot of options. Reference the docs below:   
https://pycaret.readthedocs.io/en/latest/api/regression.html#module-pycaret.regression

In [28]:
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

setup_params_cls = {
    "log_experiment": True,
    "log_profile": False,
    "log_plots": False,
    "experiment_name": f"CLS_1_{timestamp}",
    "data": training_df,
    "test_data": testing_df,
    "target": "CLS_TARGET",
    "preprocess": False,
    "normalize": False,  # zscore
    "transformation": False,  # yeo-johnson power transform to make data more Gaussian
    "remove_outliers": False,  # using SVD
    "remove_multicollinearity": False,
    "polynomial_features": False,
    "feature_selection": False,
    "pca": False,
    "pca_components": 10,
    "numeric_features": [],
    "ignore_features": ["game_id"],
}

In [29]:
py_cls.setup(**setup_params_cls)

,Description,Value
0,Session id,4086
1,Target,CLS_TARGET
2,Target type,Binary
3,Original data shape,"(3629, 3)"
4,Transformed data shape,"(3629, 2)"
5,Transformed train set shape,"(2399, 2)"
6,Transformed test set shape,"(1230, 2)"
7,Ignore features,1
8,Numeric features,1


2023/08/20 20:02:33 INFO mlflow.tracking.fluent: Experiment with name 'CLS_1_20230820200232' does not exist. Creating a new experiment.


<a id=compare_cls></a>

### Compare Models

In [30]:
best_model_cls = py_cls.compare_models(turbo=False, sort="Accuracy")

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
mlp,MLP Classifier,0.5273,0.5163,0.2915,0.5240,0.3637,0.0363,0.0443,0.3170
nb,Naive Bayes,0.5211,0.5220,0.0000,0.0000,0.0000,0.0000,0.0000,0.1350
dummy,Dummy Classifier,0.5211,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.1780
qda,Quadratic Discriminant Analysis,0.5211,0.5220,0.0000,0.0000,0.0000,0.0000,0.0000,0.1790
rbfsvm,SVM - Radial Kernel,0.5198,0.5096,0.3812,0.4963,0.4295,0.0285,0.0288,0.6750
lr,Logistic Regression,0.5194,0.5070,0.0087,0.1708,0.0165,-0.0026,-0.0073,0.4850
ridge,Ridge Classifier,0.5194,0.0000,0.0087,0.1708,0.0165,-0.0026,-0.0073,0.1800
lda,Linear Discriminant Analysis,0.5194,0.5070,0.0087,0.1708,0.0165,-0.0026,-0.0073,0.1850
gpc,Gaussian Process Classifier,0.5135,0.5193,0.3541,0.4875,0.4094,0.0142,0.0143,5.0630
gbc,Gradient Boosting Classifier,0.5098,0.5098,0.4134,0.4851,0.4450,0.0118,0.0117,0.3450


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

In [31]:
print(best_model_cls)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=4086, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)


<a id=create_cls></a>

### Create Selected Model

In [32]:
model_cls = py_cls.create_model("nb")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5208,0.4326,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.5208,0.5295,0.0000,0.0000,0.0000,0.0000,0.0000
2,0.5208,0.5433,0.0000,0.0000,0.0000,0.0000,0.0000
3,0.5208,0.5638,0.0000,0.0000,0.0000,0.0000,0.0000
4,0.5208,0.5443,0.0000,0.0000,0.0000,0.0000,0.0000
5,0.5208,0.5060,0.0000,0.0000,0.0000,0.0000,0.0000
6,0.5208,0.4938,0.0000,0.0000,0.0000,0.0000,0.0000
7,0.5208,0.5194,0.0000,0.0000,0.0000,0.0000,0.0000
8,0.5208,0.5773,0.0000,0.0000,0.0000,0.0000,0.0000


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

<a id=tune_cls></a>

### Tune Selected Model

In [33]:
tuned_model_cls = py_cls.tune_model(model_cls)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5208,0.4326,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.5208,0.5295,0.0000,0.0000,0.0000,0.0000,0.0000
2,0.5208,0.5433,0.0000,0.0000,0.0000,0.0000,0.0000
3,0.5208,0.5638,0.0000,0.0000,0.0000,0.0000,0.0000
4,0.5208,0.5443,0.0000,0.0000,0.0000,0.0000,0.0000
5,0.5208,0.5060,0.0000,0.0000,0.0000,0.0000,0.0000
6,0.5208,0.4938,0.0000,0.0000,0.0000,0.0000,0.0000
7,0.5208,0.5194,0.0000,0.0000,0.0000,0.0000,0.0000
8,0.5208,0.5773,0.0000,0.0000,0.0000,0.0000,0.0000


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [34]:
print(tuned_model_cls)

GaussianNB(priors=None, var_smoothing=1e-09)


In [35]:
model_report["details"] = tuned_model_cls.get_params()

<a id=evaluate_cls></a>

### Evaluate Model

https://pycaret.readthedocs.io/en/latest/api/regression.html#pycaret.regression.evaluate_model

In [36]:
py_cls.evaluate_model(tuned_model_cls)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

https://pycaret.readthedocs.io/en/latest/api/regression.html#pycaret.regression.interpret_model

In [37]:
# py_cls.interpret_model(tuned_model_cls)

In [38]:
train_predictions_cls = py_cls.predict_model(tuned_model_cls, data=training_df)
train_prediction_metrics = py_cls.pull()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Naive Bayes,0.5211,0.5273,0.0000,0.0000,0.0000,0.0000,0.0000


In [39]:
model_report["train_accuracy"] = train_prediction_metrics["Accuracy"][0]
model_report["train_auc"] = train_prediction_metrics["AUC"][0]

In [40]:
test_predictions_cls = py_cls.predict_model(tuned_model_cls, data=testing_df)
test_prediction_metrics = py_cls.pull()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Naive Bayes,0.4976,0.4952,0.0000,0.0000,0.0000,0.0000,0.0000


In [41]:
model_report["test_accuracy"] = test_prediction_metrics["Accuracy"][0]
model_report["test_auc"] = test_prediction_metrics["AUC"][0]

In [42]:
roi_results_cls = calculate_roi(
    test_predictions_cls, "CLS_TARGET", "prediction_label", pred_prob="prediction_score"
)
roi_results_cls

,Label,Total ROI,Average ROI per Bet
0,"All Bets, Even Amount",-600.00000,-0.49
1,"All Bets, Typical Odds",-6108.00000,-4.97
2,"Cutoff 50% Bets, Even Odds",-600.00000,-0.49
3,"Cutoff 50% Bets, Typical Odds",-6108.00000,-4.97
4,"Cutoff 55% Bets, Even Odds",200.00000,3.57
5,"Cutoff 55% Bets, Typical Odds",-61.00000,-1.09
6,"Cutoff 60% Bets, Even Odds",100.00000,100.00
7,"Cutoff 60% Bets, Typical Odds",91.00000,91.00
8,"Cutoff 65% Bets, Even Odds",0.00000,0.00
9,"Cutoff 65% Bets, Typical Odds",0.00000,0.00


In [43]:
model_report["roi_all_bets_even_amount_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "All Bets, Even Amount"
]["Average ROI per Bet"].iloc[0]

model_report["roi_all_bets_typical_odds_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "All Bets, Typical Odds"
]["Average ROI per Bet"].iloc[0]

model_report["roi_cutoff_50_even_odds_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "Cutoff 50% Bets, Even Odds"
]["Average ROI per Bet"].iloc[0]

model_report["roi_cutoff_50_typical_odds_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "Cutoff 50% Bets, Typical Odds"
]["Average ROI per Bet"].iloc[0]

model_report["roi_cutoff_55_even_odds_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "Cutoff 55% Bets, Even Odds"
]["Average ROI per Bet"].iloc[0]

model_report["roi_cutoff_55_typical_odds_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "Cutoff 55% Bets, Typical Odds"
]["Average ROI per Bet"].iloc[0]

model_report["roi_cutoff_60_even_odds_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "Cutoff 60% Bets, Even Odds"
]["Average ROI per Bet"].iloc[0]

model_report["roi_cutoff_60_typical_odds_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "Cutoff 60% Bets, Typical Odds"
]["Average ROI per Bet"].iloc[0]

model_report["roi_cutoff_65_even_odds_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "Cutoff 65% Bets, Even Odds"
]["Average ROI per Bet"].iloc[0]

model_report["roi_cutoff_65_typical_odds_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "Cutoff 65% Bets, Typical Odds"
]["Average ROI per Bet"].iloc[0]

model_report["roi_cutoff_70_even_odds_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "Cutoff 70% Bets, Even Odds"
]["Average ROI per Bet"].iloc[0]

model_report["roi_cutoff_70_typical_odds_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "Cutoff 70% Bets, Typical Odds"
]["Average ROI per Bet"].iloc[0]

model_report["roi_all_bets_even_amount_kelly_criterion_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "All Bets, Even Amount, Kelly Criterion"
]["Average ROI per Bet"].iloc[0]

model_report["roi_all_bets_typical_odds_kelly_criterion_avg"] = roi_results_cls[
    roi_results_cls["Label"] == "All Bets, Typical Odds, Kelly Criterion"
]["Average ROI per Bet"].iloc[0]

<a id=finalize_and_store_cls></a>

### Model Finalization and Storage

In [44]:
final_model_cls = py_cls.finalize_model(tuned_model_cls)

In [45]:
platform = "pycaret"
problem_type = "cls"
model_type = "nb"
datetime_str = model_report["datetime"].strftime("%Y_%m_%d_%H_%M_%S")

model_id = f"{platform}_{problem_type}_{model_type}_{datetime_str}"
model_id

'pycaret_cls_nb_2023_08_20_20_02_32'

In [46]:
py_cls.save_model(final_model_cls, f"../models/AutoML/{model_id}")

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
          steps=[('placeholder', None),
                 ('actual_estimator',
                  GaussianNB(priors=None, var_smoothing=1e-09))],
          verbose=False),
 '../models/AutoML/pycaret_cls_nb_2023_08_20_20_02_32.pkl')

In [47]:
model_report["platform"] = platform
model_report["model_type"] = model_type
model_report["model_id"] = model_id

In [48]:
model_report

{'datetime': Timestamp('2023-08-20 20:02:32.195308'),
 'problem_type': 'cls',
 'target': 'CLS_TARGET',
 'features': ['open_line'],
 'training_start_date': Timestamp('2020-12-22 00:00:00'),
 'training_end_date': Timestamp('2022-04-10 00:00:00'),
 'testing_start_date': Timestamp('2022-10-18 00:00:00'),
 'testing_end_date': Timestamp('2023-04-09 00:00:00'),
 'training_df_shape': (2399, 3),
 'testing_df_shape': (1230, 3),
 'ind_baseline_train': 0.47894956231763236,
 'ind_baseline_test': 0.5024390243902439,
 'dep_baseline_train': 0.5210504376823677,
 'dep_baseline_test': 0.4975609756097561,
 'details': {'priors': None, 'var_smoothing': 1e-09},
 'train_accuracy': 0.5211,
 'train_auc': 0.5273,
 'test_accuracy': 0.4976,
 'test_auc': 0.4952,
 'roi_all_bets_even_amount_avg': -0.49,
 'roi_all_bets_typical_odds_avg': -4.97,
 'roi_cutoff_50_even_odds_avg': -0.49,
 'roi_cutoff_50_typical_odds_avg': -4.97,
 'roi_cutoff_55_even_odds_avg': 3.57,
 'roi_cutoff_55_typical_odds_avg': -1.09,
 'roi_cutoff_60

In [49]:
save_model_report(model_report)